In [100]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow import keras

assert tf.__version__ >= "2.0"

# To make the output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

AUTOTUNE=tf.data.experimental.AUTOTUNE

In [65]:
TOTAL_FEATURES_COUNT = 12410
TOTAL_COLUMNS_SIZE = 365
PON_COUNT_2019 = {
    'total': 101020,
    'yes': 28462,
    'no': 72558
}
dataset_path = '../dataset'

In [66]:
def load_data(csv_file):
    csv_path = os.path.join(dataset_path, csv_file)
    return pd.read_csv(csv_path, sep=',')

In [106]:
def get_label(image_path):
    image_name = image_path.numpy().decode('utf-8').split('\\')[-1]
    image_index = int(image_name[9:-4])
    return tf.constant(labels['label'][image_index - 1])

In [68]:
def decode_image(image):
    image = tf.image.decode_png(image, channels=1)  # grayscale
    image = tf.image.convert_image_dtype(image, tf.float32) 
    return image

decode_img(tf.io.read_file(os.path.join(dataset_path, 'pon_2019/pon_2019_1.png')))

<tf.Tensor: shape=(34, 365, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>

In [113]:
type(decode_img(tf.io.read_file(os.path.join(dataset_path, 'pon_2019/pon_2019_1.png'))))

tensorflow.python.framework.ops.EagerTensor

In [111]:
def process_path(file_path):
    label = get_label(file_path)
    image = tf.io.read_file(file_path)
    image = decode_img(image)
    return image, label

In [70]:
path, dirs, files = next(os.walk(os.path.join(dataset_path, 'pon_2019')))
assert len(files) == PON_COUNT_2019['total']

with open(os.path.join(dataset_path, 'pon_actions_2019.csv')) as f:
    f.readline()
    assert sum(1 for line in f) == PON_COUNT_2019['total']
    f.seek(0)
    f.readline()
    assert sum(int(line[-2]) for line in f) == PON_COUNT_2019['yes']

print('Dataset OK')

Dataset OK


In [130]:
image_files = tf.data.Dataset.list_files(os.path.join(dataset_path, 'pon_2019/*.png'))

for f in image_files.take(2):
    print(f)
    print(process_path(f))
    print(type(process_path(f)))

tf.Tensor(b'..\\dataset\\pon_2019\\pon_2019_35081.png', shape=(), dtype=string)
(<tf.Tensor: shape=(34, 365, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
<class 'tuple'>
tf.Tensor(b'..\\dataset\\pon_2019\\pon_2019_95389.png', shape=(), dtype=string)
(<tf.Tensor: shape=(34, 365, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [0.],
   

In [74]:
labels = load_data('pon_actions_2019.csv')

labels.head()

,image,label
0,pon_2019_1.png,0
1,pon_2019_2.png,0
2,pon_2019_3.png,1
3,pon_2019_4.png,0
4,pon_2019_5.png,1


In [127]:
labels['label'][99999]

1

In [75]:
for i in range(len(labels)):
    assert labels['image'][i] == 'pon_2019_' + str(i + 1) + '.png'

print('Dataframe OK')

Dataframe OK


In [137]:
X = []
y = []

for f in image_files:
    image, label = process_path(f)
    X.append(image)
    y.append(label)

# X = tf.stack(X)
# y = tf.stack(y)

In [138]:
X

[<tf.Tensor: shape=(34, 365, 1), dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        ...,
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]], dtype=float32)>,
 <tf.Tensor: shape=(34, 365, 1), dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
   

In [139]:
y

[<tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64

In [141]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1,
                                                  train_size=0.9,
                                                  random_state=42,
                                                  stratify=y)
X_train = tf.stack(X_train)
X_dev = tf.stack(X_dev)
y_train = tf.stack(y_train)
y_dev = tf.stack(y_dev)
print('X_train shape:', X_train.shape)
print('X_dev.shape:', X_dev.shape)
print('y_train shape:', y_train.shape)
print('y_dev.shape:', y_dev.shape)

X_train shape: (90918, 34, 365, 1)
X_dev.shape: (10102, 34, 365, 1)
y_train shape: (90918,)
y_dev.shape: (10102,)


In [144]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3,
                        activation='relu',
                        padding="VALID")

tf.keras.backend.clear_session()

model = keras.models.Sequential([
    DefaultConv2D(filters=128, input_shape=[34, 365, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    DefaultConv2D(filters=128),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    DefaultConv2D(filters=128),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=2, activation='softmax'),
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 363, 128)      1280      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 363, 128)      512       
_________________________________________________________________
dropout (Dropout)            (None, 32, 363, 128)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 361, 128)      147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 361, 128)      512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 361, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 359, 128)      1

In [146]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=2, validation_data=(X_dev, y_dev))

Epoch 1/2


ResourceExhaustedError:  OOM when allocating tensor with shape[3200,32,363,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential/conv2d/Relu (defined at <ipython-input-146-aa2898b5c747>:5) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1421678]

Function call stack:
train_function
